# Recomendando livros

# 1- Carregando e limpando dados

In [14]:
import pandas as pd

In [17]:
import pandas as pd

# on_bad_lines='warn' vai pular as linhas com erro e te avisar quais são
df_livros = pd.read_csv("books.csv", on_bad_lines='warn')

# Verificando se carregou
print(f"Total de livros carregados: {len(df_livros)}")
print(df_livros.info())

Total de livros carregados: 11123
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB
None


/tmp/ipykernel_527282/19923291.py:4: ParserWarning: Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13

  df_livros = pd.read_csv("books.csv", on_bad_lines='warn')


In [18]:
print(df_livros.shape)

(11123, 12)


In [19]:
# 1. Quais são os 10 autores com mais livros no dataset?
print("--- Autores mais produtivos ---")
print(df_livros['authors'].value_counts().head(10))


--- Autores mais produtivos ---
authors
Stephen King                   40
P.G. Wodehouse                 40
Rumiko Takahashi               39
Orson Scott Card               35
Agatha Christie                33
Piers Anthony                  30
Mercedes Lackey                29
Sandra Brown                   29
Dick Francis                   28
Margaret Weis/Tracy Hickman    23
Name: count, dtype: int64


In [20]:

# 2. Quais são os 10 livros com as maiores notas (apenas os que têm muitas avaliações)?
# Isso evita que um livro com apenas 1 voto nota 5 apareça em primeiro.
print("\n--- Livros mais bem avaliados (com mais de 10.000 votos) ---")
populares_e_bons = df_livros[df_livros['ratings_count'] > 10000]
print(populares_e_bons.sort_values(by='average_rating', ascending=False)[['title', 'authors', 'average_rating']].head(10))



--- Livros mais bem avaliados (com mais de 10.000 votos) ---
                                                  title  \
6587                     The Complete Calvin and Hobbes   
4     Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
6589      It's a Magical World (Calvin and Hobbes  #11)   
6          Harry Potter Collection (Harry Potter  #1-6)   
6590  Homicidal Psycho Jungle Cat (Calvin and Hobbes...   
6591                           The Days Are Just Packed   
6294                   The Absolute Sandman  Volume One   
6588       The Calvin and Hobbes Tenth Anniversary Book   
72    The Lord of the Rings: The Art of the Fellowsh...   
21    J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...   

                                                authors  average_rating  
6587                                     Bill Watterson            4.82  
4                            J.K. Rowling/Mary GrandPré            4.78  
6589                                     Bill Watterson           

In [24]:
# Remove espaços extras de TODOS os nomes de colunas
df_livros.columns = df_livros.columns.str.strip()

# Agora você pode testar se funcionou
print(df_livros.columns.tolist())

['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13', 'language_code', 'num_pages', 'ratings_count', 'text_reviews_count', 'publication_date', 'publisher']


In [26]:

# 3. Existe correlação entre o número de páginas e a nota?

print(df_livros[['num_pages', 'average_rating']].corr())

                num_pages  average_rating
num_pages        1.000000        0.150477
average_rating   0.150477        1.000000


In [27]:
print(df_livros.info())

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7   num_pages           11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usag

# 2- Sistema de recomendação

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. Preparação: Vamos criar uma 'sopa de palavras' combinando título e autor
# Isso ajuda o algoritmo a entender o contexto completo
df_livros['metadata'] = df_livros['title'] + " " + df_livros['authors'] + " " + df_livros['publisher']

# 2. Transformação: Convertendo texto em números
tfidf = TfidfVectorizer(stop_words='english') # Ignora palavras comuns como 'the', 'and'
tfidf_matrix = tfidf.fit_transform(df_livros['metadata'])

# 3. Cálculo de Similaridade: Vamos ver o quão perto cada vetor está do outro (Similaridade de Cosseno)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 4. Função de Recomendação
def recomendar_livros(titulo, cosine_sim=cosine_sim):
    try:
        # Pega o índice do livro que bate com o título
        idx = df_livros.index[df_livros['title'] == titulo][0]

        # Pega as notas de similaridade desse livro com todos os outros
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Ordena do mais parecido para o menos parecido
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Pega os 5 livros mais parecidos (excluindo ele mesmo)
        sim_scores = sim_scores[1:6]

        # Pega os índices e retorna os títulos
        livros_indices = [i[0] for i in sim_scores]
        return df_livros['title'].iloc[livros_indices]
    except:
        return "Livro não encontrado no dataset. Verifique se o título está idêntico."

# --- TESTE ---
print("Se você gostou de 'The Hobbit', também pode gostar de:")
print(recomendar_livros('Dune (Dune #1)'))

Se você gostou de 'The Hobbit', também pode gostar de:
69       Heretics of Dune (Dune Chronicles  #5)
71        Heretics of Dune (Dune Chronicles #5)
67            Dune Messiah (Dune Chronicles #2)
66      Chapterhouse: Dune (Dune Chronicles #6)
65    God Emperor of Dune (Dune Chronicles  #4)
Name: title, dtype: object


# 3- Sistema de recomendação melhorado

In [34]:
def recomendar_livros_premium(termo_busca, nota_minima=4.0, avaliacoes_minimas=500):
    try:
        # Busca flexível: encontra o primeiro livro que contém o termo digitado
        mask = df_livros['title'].str.contains(termo_busca, case=False, na=False)
        if not mask.any():
            return f"Nenhum livro encontrado contendo '{termo_busca}'."
        
        # Pega o índice do primeiro livro encontrado
        idx = df_livros[mask].index[0]
        titulo_real = df_livros.loc[idx, 'title']
        print(f"Livro base encontrado: {titulo_real}\n")

        # Cálculo de similaridade
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        recomendacoes_validas = []
        
        for i, score in sim_scores[1:]:
            livro = df_livros.iloc[i]
            
            # Filtros (corrigido para 'livro')
            if livro['average_rating'] >= nota_minima and livro['ratings_count'] >= avaliacoes_minimas:
                recomendacoes_validas.append({
                    'Título': livro['title'],
                    'Autor': livro['authors'],
                    'Nota': livro['average_rating'],
                    'Avaliações': livro['ratings_count'],
                    'Similaridade': round(score, 2) # Adicionamos o grau de semelhança
                })
            
            if len(recomendacoes_validas) == 5:
                break
        
        return pd.DataFrame(recomendacoes_validas)

    except Exception as e:
        return f"Erro inesperado: {e}"

# --- TESTE COM BUSCA PARCIAL ---
# Agora você não precisa do título idêntico, apenas um trecho dele
resultado = recomendar_livros_premium('Hobbit', nota_minima=4.0, avaliacoes_minimas=500)
print(resultado)

Livro base encontrado: J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings

                                              Título           Autor  Nota  \
0                                         The Hobbit  J.R.R. Tolkien  4.27   
1  The Lord of the Rings (The Lord of the Rings  ...  J.R.R. Tolkien  4.50   
2  The Lord of the Rings (The Lord of the Rings  ...  J.R.R. Tolkien  4.50   
3  The Return of the King (The Lord of the Rings ...  J.R.R. Tolkien  4.53   
4                The Hobbit  or There and Back Again  J.R.R. Tolkien  4.27   

   Avaliações  Similaridade  
0        3213          0.56  
1         682          0.52  
2        1710          0.49  
3        1635          0.49  
4     2530894          0.48  


In [36]:
import gradio as gr

def interface_recomendacao(nome_do_livro, nota_min, avaliacoes_min):
    # Chamamos a função que criamos anteriormente
    # Note: ela precisa estar definida no seu notebook ou script
    resultado = recomendar_livros_premium(nome_do_livro, nota_min, avaliacoes_min)
    
    # Se o resultado for uma string (mensagem de erro), retornamos ela
    if isinstance(resultado, str):
        return resultado
    
    # Se for um DataFrame, o Gradio exibe como uma tabela bonita
    return resultado

# Criando a interface visual
app = gr.Interface(
    fn=interface_recomendacao, 
    inputs=[
        gr.Textbox(label="Digite o título do livro (ou parte dele)", placeholder="Ex: Hobbit"),
        gr.Slider(minimum=0, maximum=5, value=4.0, step=0.1, label="Nota Mínima"),
        gr.Number(value=500, label="Mínimo de Avaliações")
    ],
    outputs=gr.Dataframe(label="Livros Recomendados"),
    title="📚 Meu Sistema de Recomendação de Livros",
    description="Projeto de Portfólio: Digite um livro que você gostou e veja sugestões de alta qualidade.",
    theme="huggingface" # Um tema elegante e limpo
)

# Lançar a interface
app.launch()

/home/fernando/venv/lib/python3.13/site-packages/gradio/interface.py:171: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  super().__init__(
/home/fernando/venv/lib/python3.13/site-packages/gradio/utils.py:573: UserWarning: Cannot load huggingface. Caught Exception: Client error '404 Not Found' for url 'https://huggingface.co/api/spaces/huggingface' (Request ID: Root=1-699e42a9-4ed6382059dfc86630c3d8c6;96125fb1-5b4a-4faa-bfc2-5539acd5412b)
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Livro base encontrado: J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings

Livro base encontrado: God Emperor of Dune (Dune Chronicles  #4)

